$\Large\textbf{Lab 4.}$ $\large\textbf{Exercise 2.}$



In [ ]:
import numpy as np 
from numpy.linalg import cond
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt

In [ ]:
#Now we will define a Python function which will compute and return the objective function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return np.sqrt(x[0]**2+4) + np.sqrt(x[1]**2+4)

In [ ]:
#Now we will define a Python function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  return np.array([x[0]/np.sqrt(x[0]**2+4),x[1]/np.sqrt(x[1]**2+4)])

In [ ]:
#method to find Hessian matrix: Complete the code
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[4/np.power(x[0]**2+4,3/2),0],[0,4/np.power(x[1]**2+4,3/2)]])

In [ ]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  a=1/(evalh(x)[0][0])
  b=1/(evalh(x)[1][1])
  return np.array([[a, 0], [0, b]])


In [ ]:
def compute_D_k_newton(x):
    assert type(x) is np.ndarray
    assert len(x) == 2
    
    return np.linalg.inv(evalh(x))

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_exact(x, gradf, A, b): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(A) is np.ndarray and A.shape[0] == 2 and  A.shape[1] == 2 #allow only a 2x2 array
   
  #Complete the code 
  num = np.dot(np.dot(x.T, A), gradf) + np.dot(np.dot(gradf.T, A), x) + 2*np.dot(b.T, gradf)
  den = 2*np.dot(np.dot(gradf.T, A), gradf)
  step_length = num/den
    
  return step_length 

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma, *args): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  #Complete the code 
  alpha = alpha_start
  pk = -gradf
  while evalf(np.add(x, alpha*pk)) > np.add(evalf(x), gamma*alpha*np.dot(gradf.T,pk)):
    alpha = rho*alpha

  return alpha
  

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma, *args): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  
  #Complete the code 
  alpha = alpha_start
  pk = -gradf
  while evalf(np.add(x, alpha*np.dot(direction,pk))) > np.subtract(evalf(x), gamma*alpha*np.dot(np.dot(direction,gradf), gradf)):
    alpha = rho*alpha
  return alpha

In [ ]:
#line search type 
CONSTANT_STEP_LENGTH = 1
BACKTRACKING_LINE_SEARCH = 2

# D_k choice type
newton = 3
diagonal_approx = 4

In [ ]:
# Code for gradient descent with scaling to find the minimizer without scaling

def find_minimizer_gd(start_x, tol, line_search_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)

    #initialization for backtracking line search
    if(line_search_type == BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
    
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 

        print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    #print(k)
    return x, evalf(x), k
 

In [ ]:
# Code for gradient descent with scaling to find the minimizer with scaling

def find_minimizer_gdscaling(start_x, tol, line_search_type, D_k_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)
    
    #initialization for backtracking line search
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
        if D_k_type == newton:
            d_k = compute_D_k_newton(x)
        elif D_k_type == diagonal_approx:
            d_k = compute_D_k(x)
        else:
            raise ValueError("Invalid argument.")


        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg(x) 
        print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))


    return x, evalf(x), k
 

Code for answer 2

In [ ]:
my_start_x = np.array([2.,2.])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [ ]:
#check gradient descent with backtracking line search 
x_opt_bls, f_bls, iter_bls = find_minimizer_gdscaling(my_start_x, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls)
print("Minimum function value =", f_bls)
print("No of iterations =", iter_bls)

iter: 1  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 2  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 3  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 4  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 5  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 6  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 7  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 8  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 9  x: [-2. -2.]  f(x): 5.656854249492381  grad at 

KeyboardInterrupt: ignored

In [ ]:
#check gradient descent with scaling and backtracking line search 
x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

iter: 1  x: [0. 0.]  f(x): 4.0  grad at x: [0. 0.]  gradient norm: 0.0
Minimizer = [0. 0.]
Minimum function value = 4.0
No of iterations = 1


Code for answer 3

In [ ]:
my_start_x = np.array([2.0,2.0])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [ ]:
#check gradient descent with backtracking line search 
x_opt_bls, f_bls, iter_bls = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls)
print("Minimum function value =", f_bls)
print("No of iterations =", iter_bls)

iter: 1  x: [1.29289322 1.29289322]  f(x): 4.763012859631521  grad at x: [0.54288882 0.54288882]  gradient norm: 0.7677607340693352
iter: 2  x: [0.7500044 0.7500044]  f(x): 4.272004960744186  grad at x: [0.35112525 0.35112525]  gradient norm: 0.4965660856779933
iter: 3  x: [0.39887915 0.39887915]  f(x): 4.078776570026804  grad at x: [0.19558764 0.19558764]  gradient norm: 0.276602699517397
iter: 4  x: [0.20329151 0.20329151]  f(x): 4.020610618589563  grad at x: [0.10112469 0.10112469]  gradient norm: 0.14301191174902647
iter: 5  x: [0.10216681 0.10216681]  f(x): 4.005215628575656  grad at x: [0.05101689 0.05101689]  gradient norm: 0.07214877156754755
iter: 6  x: [0.05114993 0.05114993]  f(x): 4.001307943737469  grad at x: [0.0255666 0.0255666]  gradient norm: 0.036156638370264234
iter: 7  x: [0.02558332 0.02558332]  f(x): 4.000327239848125  grad at x: [0.01279062 0.01279062]  gradient norm: 0.018088662064556508
iter: 8  x: [0.01279271 0.01279271]  f(x): 4.000081825857272  grad at x: [0

Code for answer 4

In [ ]:
my_start_x2 = np.array([8.0,8.0])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [ ]:
#check gradient descent with constant step length 
x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gdscaling(my_start_x2, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

iter: 1  x: [-128. -128.]  f(x): 256.03124809288414  grad at x: [-0.99987795 -0.99987795]  gradient norm: 1.414040960485301
iter: 2  x: [524288. 524288.]  f(x): 1048576.0000076294  grad at x: [1. 1.]  gradient norm: 1.4142135623628054
iter: 3  x: [-3.6028797e+16 -3.6028797e+16]  f(x): 7.205759403792794e+16  grad at x: [-1. -1.]  gradient norm: 1.4142135623730951
iter: 4  x: [1.16920131e+49 1.16920131e+49]  f(x): 2.3384026197294447e+49  grad at x: [1. 1.]  gradient norm: 1.4142135623730951
iter: 5  x: [-3.99583814e+146 -3.99583814e+146]  f(x): 7.99167628880894e+146  grad at x: [-1. -1.]  gradient norm: 1.4142135623730951


<ipython-input-4-9ed0219b9ace>:5: RuntimeWarning: overflow encountered in power
  return np.array([[4/np.power(x[0]**2+4,3/2),0],[0,4/np.power(x[1]**2+4,3/2)]])


LinAlgError: ignored

In [ ]:

x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gdscaling(my_start_x2, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

iter: 1  x: [-0.5 -0.5]  f(x): 4.123105625617661  grad at x: [-0.24253563 -0.24253563]  gradient norm: 0.3429971702850177
iter: 2  x: [-0.234375 -0.234375]  f(x): 4.027372165879384  grad at x: [-0.11639103 -0.11639103]  gradient norm: 0.16460177506779788
iter: 3  x: [-0.11557817 -0.11557817]  f(x): 4.006673590120265  grad at x: [-0.05769283 -0.05769283]  gradient norm: 0.08158998647858538
iter: 4  x: [-0.0575961 -0.0575961]  f(x): 4.001658311393147  grad at x: [-0.02878611 -0.02878611]  gradient norm: 0.04070971277400298
iter: 5  x: [-0.02877417 -0.02877417]  f(x): 4.000413954866833  grad at x: [-0.01438559 -0.01438559]  gradient norm: 0.020344301811841013
iter: 6  x: [-0.0143841 -0.0143841]  f(x): 4.000103449894273  grad at x: [-0.00719187 -0.00719187]  gradient norm: 0.010170834833290178
iter: 7  x: [-0.00719168 -0.00719168]  f(x): 4.000025860048939  grad at x: [-0.00359582 -0.00359582]  gradient norm: 0.005085253008744151
iter: 8  x: [-0.00359579 -0.00359579]  f(x): 4.00000646486072

Code for answer 5

In [ ]:
#check gradient descent with scaling and backtracking line search 
x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gd(my_start_x2, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

iter: 1  x: [7.0298575 7.0298575]  f(x): 14.617646386236455  grad at x: [0.96183165 0.96183165]  gradient norm: 1.3602353696564338
iter: 2  x: [6.06802585 6.06802585]  f(x): 12.778253036663257  grad at x: [0.94974263 0.94974263]  gradient norm: 1.3431389132092362
iter: 3  x: [5.11828321 5.11828321]  f(x): 10.990327209300977  grad at x: [0.93141598 0.93141598]  gradient norm: 1.3172211158287075
iter: 4  x: [4.18686723 4.18686723]  f(x): 9.280055429474341  grad at x: [0.90233669 0.90233669]  gradient norm: 1.2760967785481656
iter: 5  x: [3.28453054 3.28453054]  f(x): 7.69107037829689  grad at x: [0.85411533 0.85411533]  gradient norm: 1.2079014785015503
iter: 6  x: [2.43041522 2.43041522]  f(x): 6.295051430162643  grad at x: [0.77216691 0.77216691]  gradient norm: 1.0920089216846292
iter: 7  x: [1.6582483 1.6582483]  f(x): 5.19607060600531  grad at x: [0.63827012 0.63827012]  gradient norm: 0.9026502594886293
iter: 8  x: [1.01997818 1.01997818]  f(x): 4.490147211632515  grad at x: [0.454

Answer 2) 

**Newton's method (with constant step length of 1)**

Does not converge

**Newton's method (with Backtracking line search)**

Minimizer = [0. 0.]

Minimum function value = 4.0

No of iterations = 1

**Observations:** 

The Newton's method with constant step length does not converge as the minimizer keeps oscillating between (2,2) and (-2,-2) whereas Newton's method with backtracking line search yields the exact minimizer (0,0). The minimum objective function value at the obtained minimizer using Newton's method with backtracking line search is 4 and number of iterations is 1.

Answer 3) 

**Gradient Descent (with backtracking line search without scaling)**

Minimizer = [7.62525638e-10 7.62525638e-10]

Minimum function value = 4.0

No of iterations = 32

**Observations:** 
The Newton's method with constant step length does not converge as the minimizer keeps oscillating between (2,2) and (-2,-2) whereas Newton's method with backtracking line search yields the exact minimizer (0,0). Newton's method with backtracking line search takes 1 iteration to converge whereas Gradient Descent with backtracking line search without scaling takes 32 iterations to converge. Minimum fucntion value in both is 4 but in gradient descent (with backtracking line search without scaling), it is [7.62525638e-10, 7.62525638e-10].

Answer 4) 

**Newton's method (with constant step length of 1)**

It shows an error "singular matrix"

**Newton's method (with Backtracking line search)**

Minimizer = [2.83764947e-12 2.83764947e-12]

Minimum function value = 4.0

No of iterations = 13

**Observations:** 

With Newton's method (with constant step length of 1), it does not converge and the minimum function value diverges to infinity. After 5 iterations, the code stopped running and the minimizer shown is nowhere close to the actual minimizer [0, 0]. But, Newton's method with backtracking line search gives the minimum function value as 4 in 13 iterations.

Answer 5) 

**Gradient Descent (without scaling) with backtracking line search**

Minimizer = [8.3177047e-10 8.3177047e-10]

Minimum function value = 4.0

No of iterations = 39


**Observations:** 

With Newton's method (with constant step length of 1), it does not converge and the minimum function value diverges to infinity. After 5 iterations, the code stopped running and the minimizer shown is nowhere close to the actual minimizer [0, 0]. The minimum function value is both Newton's method with backtracking line search and Gradient descent (with backtracking line search without scaling) is 4. The former gives a better result in terms of minimizer and minimum function value being close to the actual minimizer and minimum function value and in less iterations (13) than the latter (39).